In [1]:
import os
os.chdir('/Users/victormanuelfreixaslemus/Desktop/Projects/Git_portafolio/STO_Gaussian_overlaps/STO_Gaussian_overlaps')
from src.molden import read_molden
from src.sto_params import get_sto_params
from src.overlap.api import overlap_sto_gto

In [ ]:
# Testing molden file reader

moldenFile = "examples/benzene_cart.mld"

mol = read_molden(moldenFile)

print(f"Units: {mol.units}")
print(f"# atoms: {len(mol.atoms)}")
print(f"First atom: {mol.atoms[0].element} @ {mol.atoms[0].coords}")

# shells for atom 1
for sh in mol.shells_by_atom[0]:
    print(sh.L, sh.scale, "with", len(sh.primitives), "primitives")
    for prim in sh.primitives:
        print("  alpha:", prim.alpha, "coeffs:", prim.coeffs)

Units: AU
# atoms: 12
First atom: C @ (0.0, 2.63805766989283, 0.0)
s 1.0 with 3 primitives
  alpha: 71.616837 coeffs: {'s': 0.15432897000916}
  alpha: 13.045096 coeffs: {'s': 0.53532814003178}
  alpha: 3.5305122 coeffs: {'s': 0.4446345400264}
s 1.0 with 3 primitives
  alpha: 2.9412494 coeffs: {'s': -0.099967230075964}
  alpha: 0.6834831 coeffs: {'s': 0.39951283030359}
  alpha: 0.2222899 coeffs: {'s': 0.70011547053201}
p 1.0 with 3 primitives
  alpha: 2.9412494 coeffs: {'p': 0.15591627210511}
  alpha: 0.6834831 coeffs: {'p': 0.60768372820466}
  alpha: 0.2222899 coeffs: {'p': 0.39195739529202}


In [3]:
# testing sto_params

params = get_sto_params("C")
for o in params.orbitals:
    if o.n == 2 and o.l == "p":
        print(o.zeta)


1.685116


In [4]:
# Testing same-center <STO 2s | GTO s> with the **valence** s shell

moldenFile = "examples/benzene_cart.mld"
mol = read_molden(moldenFile)

# Same atom for A and B (same-center test)
atom_idx = 0  # e.g., the first carbon
A = mol.atoms[atom_idx].coords
B = mol.atoms[atom_idx].coords

# Grab **both** s shells on that atom (core first, valence second)
s_shells = [sh for sh in mol.shells_by_atom[atom_idx] if sh.L.lower() == "s"]
assert len(s_shells) >= 2, "Expected core and valence s shells on carbon."

core_s, valence_s = s_shells[0], s_shells[1]

# Pick STO 2s on the same element
sto_2s = next(o for o in get_sto_params("C").orbitals if o.n == 2 and o.l == "s")

# show which shell we’re using
print("Valence s exponents & coeffs:",
      [(p.alpha, p.coeffs["s"]) for p in valence_s.primitives])

# Compute the same-center overlap using the **valence** s shell
val = overlap_sto_gto(sto_2s, valence_s, A, B, quad_n=128)
print("Overlap <2s|valence s> (same center) =", val)

Valence s exponents & coeffs: [(2.9412494, -0.099967230075964), (0.6834831, 0.39951283030359), (0.2222899, 0.70011547053201)]
Overlap <2s|valence s> (same center) = 0.9981298324238633


In [5]:
# same-center <STO 2p | GTO p> on one carbon
moldenFile = "examples/benzene_cart.mld"
mol = read_molden(moldenFile)

i = 0  # atom index (0 = first C)
A = B = mol.atoms[i].coords

# pick the p shell on this atom (valence p in STO-3G)
p_shell = next(sh for sh in mol.shells_by_atom[i] if sh.L.lower() == "p")

# pick STO 2p for carbon
sto_2p = next(o for o in get_sto_params("C").orbitals if o.n == 2 and o.l == "p")

axes = ("x","y","z")
M = [[overlap_sto_gto(sto_2p, p_shell, A, B, axis_sto=ax_i, axis_gto=ax_j, quad_n=128)
      for ax_j in axes] for ax_i in axes]

print("p–p 3×3 (same center):")
for row in M: print(["% .6e" % v for v in row])
print("diag =", ["% .6e" % M[k][k] for k in range(3)])

p–p 3×3 (same center):
[' 9.995769e-01', ' 0.000000e+00', ' 0.000000e+00']
[' 0.000000e+00', ' 9.995769e-01', ' 0.000000e+00']
[' 0.000000e+00', ' 0.000000e+00', ' 9.995769e-01']
diag = [' 9.995769e-01', ' 9.995769e-01', ' 9.995769e-01']


In [6]:
# same-center <STO 2p | GTO p> on one carbon, with primitive printout & norm check

moldenFile = "examples/benzene_cart.mld"
mol = read_molden(moldenFile)

# pick atom (same center test)
i = 0  # 0 = first carbon in your benzene file
A = B = mol.atoms[i].coords

# ---- pick the p shell (STO-3G has a single valence p shell per C) ----
p_shell = next(sh for sh in mol.shells_by_atom[i] if sh.L.lower() == "p")

# ---- print primitives (α, c_p) so you can verify in Mathematica ----
prims = [(p.alpha, p.coeffs.get("p")) for p in p_shell.primitives]
print("GTO p-shell primitives (alpha, c_p):")
for a, c in prims:
    print(f"  alpha = {a:.10f},  c_p = {c:.14f}")

# ---- print the STO 2p zeta used ----
sto_2p = next(o for o in get_sto_params("C").orbitals if o.n == 2 and o.l == "p")
print(f"\nSTO 2p ζ used: {sto_2p.zeta:.10f}")

# ---- (optional) contracted p-shell norm check (should be ~1) ----
# <χ_p|χ_p> = sum_{ij} c_i c_j N_p(α_i) N_p(α_j) * π^{3/2} / (α_i+α_j)^{3/2}
from math import pi
from src.overlap.constants import N_p  # adjust import path if needed
norm = 0.0
for ai, ci in prims:
    for aj, cj in prims:
        norm += ci * cj * N_p(ai) * N_p(aj) * (pi ** 1.5) / (2.0 * (ai + aj) ** 2.5)
print(f"\nContracted p-shell norm = {norm:.10f}")

# ---- build the 3x3 p–p overlap block at the same center ----
axes = ("x", "y", "z")
M = [[overlap_sto_gto(sto_2p, p_shell, A, B,
                      axis_sto=ax_i, axis_gto=ax_j, quad_n=128)
      for ax_j in axes] for ax_i in axes]

print("\np–p 3×3 (same center):")
for row in M:
    print("  ", "  ".join(f"{v: .8f}" for v in row))
print("diag =", ", ".join(f"{M[k][k]: .8f}" for k in range(3)))

GTO p-shell primitives (alpha, c_p):
  alpha = 2.9412494000,  c_p = 0.15591627210511
  alpha = 0.6834831000,  c_p = 0.60768372820466
  alpha = 0.2222899000,  c_p = 0.39195739529202

STO 2p ζ used: 1.6851160000

Contracted p-shell norm = 1.0000000000

p–p 3×3 (same center):
    0.99957690   0.00000000   0.00000000
    0.00000000   0.99957690   0.00000000
    0.00000000   0.00000000   0.99957690
diag =  0.99957690,  0.99957690,  0.99957690


In [ ]:
# same-center <STO 1s | GTO s> on a hydrogen in your benzene MOLDEN

moldenFile = "examples/benzene_cart.mld"
mol = read_molden(moldenFile)

# Pick the first H atom
i = 6
A = B = mol.atoms[i].coords

# Grab the (only) s shell on that H atom
s_shell = next(sh for sh in mol.shells_by_atom[i] if sh.L.lower() == "s")

# STO 1s for hydrogen (ensure your registry has H 1s with the ζ you want)
sto_1s = next(o for o in get_sto_params("H").orbitals if o.n == 1 and o.l == "s")

# --- print primitives to match in Mathematica ---
prims = [(p.alpha, p.coeffs.get("s")) for p in s_shell.primitives]
print("H s-shell primitives (alpha, c_s):")
for a, c in prims:
    print(f"  alpha = {a:.10f},  c_s = {c:.14f}")

# --- print STO ζ used ---
print(f"\nSTO 1s ζ used (H): {sto_1s.zeta:.10f}")

# --- contracted s-shell norm (should be ~1) ---
from math import pi
from src.overlap.constants import N_s  # (2α/π)^(3/4)

s_norm = 0.0
for ai, ci in prims:
    for aj, cj in prims:
        s_norm += ci * cj * N_s(ai) * N_s(aj) * (pi ** 1.5) / ((ai + aj) ** 1.5)
print(f"\nContracted H s-shell norm = {s_norm:.10f}")

# --- same-center overlap <1s(STO) | s(GTO contracted)> ---
val = overlap_sto_gto(sto_1s, s_shell, A, B, quad_n=128)
print(f"\nOverlap <1s|s> (same center, H) = {val:.8f}")

H s-shell primitives (alpha, c_s):
  alpha = 3.4252509100,  c_s = 0.15432897070298
  alpha = 0.6239137300,  c_s = 0.53532814243847
  alpha = 0.1688554000,  c_s = 0.44463454202535

STO 1s ζ used (H): 1.8807800000

Contracted H s-shell norm = 1.0000000000

Overlap <1s|s> (same center, H) = 0.93702196


In [8]:
# Different-center <STO 2s | GTO p> between two carbons

moldenFile = "examples/benzene_cart.mld"
mol = read_molden(moldenFile)

# pick two different carbons (0 and 1 in your benzene)
iA, iB = 0, 1
A = mol.atoms[iA].coords
B = mol.atoms[iB].coords

# STO 2s on carbon (A)
sto_2s = next(o for o in get_sto_params("C").orbitals if o.n == 2 and o.l == "s")

# GTO p shell (contracted) on atom B
p_shell_B = next(sh for sh in mol.shells_by_atom[iB] if sh.L.lower() == "p")

# distance & direction (just for sanity prints)
Rx, Ry, Rz = (A[0]-B[0], A[1]-B[1], A[2]-B[2])
R = (Rx**2 + Ry**2 + Rz**2) ** 0.5
print(f"A (C{iA+1}) = {A},  B (C{iB+1}) = {B},  |R| = {R:.6f} bohr")

# compute the 3 components <2s@A | p_k@B>
axes = ("x","y","z")
vals = {ax: overlap_sto_gto(sto_2s, p_shell_B, A, B, axis_gto=ax, quad_n=128)
        for ax in axes}

print("s–p components (A=2s STO, B=contracted p GTO):")
for ax in axes:
    print(f"  <2s|p_{ax}> = {vals[ax]: .8e}")

A (C1) = (0.0, 2.63805766989283, 0.0),  B (C2) = (2.28467888459916, 1.31902883494641, 0.0),  |R| = 2.638104 bohr
s–p components (A=2s STO, B=contracted p GTO):
  <2s|p_x> = -3.33492922e-01
  <2s|p_y> =  1.92537684e-01
  <2s|p_z> =  0.00000000e+00


In [10]:
# Different-center <STO 2p_k (A) | GTO s_valence (B)>, valence-only

moldenFile = "examples/benzene_cart.mld"
mol = read_molden(moldenFile)

# pick two different carbons in benzene
iA, iB = 0, 1
A = mol.atoms[iA].coords
B = mol.atoms[iB].coords

# ---- STO: carbon 2p (valence) on A ----
sto_2p = next(o for o in get_sto_params("C").orbitals if o.n == 2 and o.l == "p")

# ---- GTO: pick the **valence s** shell on B (the more diffuse s) ----
s_shells_B = [sh for sh in mol.shells_by_atom[iB] if sh.L.lower() == "s"]
assert len(s_shells_B) >= 2, "Expected core and valence s shells on carbon."

# choose the s shell with smaller average exponent (more diffuse = valence)
def avg_alpha(shell): 
    return sum(p.alpha for p in shell.primitives) / len(shell.primitives)

s_valence_B = min(s_shells_B, key=avg_alpha)   # valence s
# (equivalently, s_shells_B[1] for STO-3G files like yours)

# ---- print valence s primitives on B for verification ----
print("B valence s primitives (alpha, c_s):")
for p in s_valence_B.primitives:
    print(f"  alpha = {p.alpha:.10f},  c_s = {p.coeffs['s']:.14f}")

# ---- geometry & displacement (R = A - B) ----
Rx, Ry, Rz = A[0]-B[0], A[1]-B[1], A[2]-B[2]
R = (Rx*Rx + Ry*Ry + Rz*Rz) ** 0.5
print(f"\nA (C{iA+1}) = {A}\nB (C{iB+1}) = {B}")
print(f"R = ({Rx:.6f}, {Ry:.6f}, {Rz:.6f}), |R| = {R:.6f} bohr")

# ---- compute valence p–s overlaps for k = x, y, z ----
for ax in ("x", "y", "z"):
    val = overlap_sto_gto(sto_2p, s_valence_B, A, B, axis_sto=ax, quad_n=128)
    print(f"<2p_{ax}@A | s_val@B> = {val: .10e}")

B valence s primitives (alpha, c_s):
  alpha = 2.9412494000,  c_s = -0.09996723007596
  alpha = 0.6834831000,  c_s = 0.39951283030359
  alpha = 0.2222899000,  c_s = 0.70011547053201

A (C1) = (0.0, 2.63805766989283, 0.0)
B (C2) = (2.28467888459916, 1.31902883494641, 0.0)
R = (-2.284679, 1.319029, 0.000000), |R| = 2.638104 bohr
<2p_x@A | s_val@B> =  3.4504069319e-01
<2p_y@A | s_val@B> = -1.9920463511e-01
<2p_z@A | s_val@B> =  0.0000000000e+00


In [11]:
# p–p overlaps: < STO 2p_k (A) | GTO p_l (B) >
moldenFile = "examples/benzene_cart.mld"
mol = read_molden(moldenFile)

# ---- choose atoms (same-center: iA==iB; different-center: iA!=iB) ----
iA, iB = 0, 1   # two carbons in your benzene (0-based)
A = mol.atoms[iA].coords
B = mol.atoms[iB].coords

# ---- STO: carbon 2p on A ----
sto_2p = next(o for o in get_sto_params("C").orbitals if o.n == 2 and o.l == "p")

# ---- GTO: p shell (contracted) on B ----
p_shell_B = next(sh for sh in mol.shells_by_atom[iB] if sh.L.lower() == "p")

# ---- print B p-shell primitives to verify (alpha, c_p) ----
print("B p-shell primitives (alpha, c_p):")
for p in p_shell_B.primitives:
    print(f"  alpha = {p.alpha:.10f},  c_p = {p.coeffs['p']:.14f}")

# ---- displacement R = A - B (sign matters for off-diagonal intuition) ----
Rx, Ry, Rz = A[0]-B[0], A[1]-B[1], A[2]-B[2]
R = (Rx*Rx + Ry*Ry + Rz*Rz) ** 0.5
print(f"\nA (C{iA+1}) = {A}\nB (C{iB+1}) = {B}")
print(f"R = ({Rx:.6f}, {Ry:.6f}, {Rz:.6f}), |R| = {R:.6f} bohr")

# ---- (optional) contracted p-shell norm at same center (should be ~1) ----
from math import pi
from src.overlap.constants import N_p
p_norm = 0.0
for pi_, ci in [(p.alpha, p.coeffs['p']) for p in p_shell_B.primitives]:
    for pj_, cj in [(p.alpha, p.coeffs['p']) for p in p_shell_B.primitives]:
        p_norm += ci*cj * N_p(pi_) * N_p(pj_) * (pi**1.5) / (2.0 * (pi_+pj_)**2.5)
print(f"\nContracted p-shell norm (B) = {p_norm:.10f}")

# ---- build 3×3 p–p block ----
axes = ("x", "y", "z")
M = [[overlap_sto_gto(sto_2p, p_shell_B, A, B,
                      axis_sto=ax_i, axis_gto=ax_j, quad_n=128)
      for ax_j in axes] for ax_i in axes]

print("\np–p 3×3:")
for row in M:
    print("  ", "  ".join(f"{v: .8e}" for v in row))
print("diag =", ", ".join(f"{M[k][k]: .8e}" for k in range(3)))

# ---- (optional) spherical-invariant comparison (trace/3) ----
trace_over_3 = sum(M[k][k] for k in range(3)) / 3.0
print(f"\n(trace/3) = {trace_over_3: .8e}")

B p-shell primitives (alpha, c_p):
  alpha = 2.9412494000,  c_p = 0.15591627210511
  alpha = 0.6834831000,  c_p = 0.60768372820466
  alpha = 0.2222899000,  c_p = 0.39195739529202

A (C1) = (0.0, 2.63805766989283, 0.0)
B (C2) = (2.28467888459916, 1.31902883494641, 0.0)
R = (-2.284679, 1.319029, 0.000000), |R| = 2.638104 bohr

Contracted p-shell norm (B) = 1.0000000000

p–p 3×3:
   -1.92825963e-01   2.38274580e-01   0.00000000e+00
    2.38274580e-01   8.23228114e-02   0.00000000e+00
    0.00000000e+00   0.00000000e+00   2.19887457e-01
diag = -1.92825963e-01,  8.23228114e-02,  2.19887457e-01

(trace/3) =  3.64614352e-02
